# Functions

In [1]:
# !python /content/MeMOTR-thesis/models/ops/setup.py build install

## MP4 functions

In [2]:
import cv2
from IPython.display import clear_output, display, HTML
from base64 import b64encode


def play_video(video_path):
    video = cv2.VideoCapture(video_path)
    while True:
        ret, frame = video.read()
        if not ret:
            break
        clear_output(wait=True)
        _, buffer = cv2.imencode('.jpg', frame)
        display(HTML('<img src="data:image/jpeg;base64,{}">'.format(b64encode(buffer).decode())))
        # time.sleep(0.1)
    video.release()

## Data transform functions

In [3]:
import torchvision.transforms.functional as F


def process_image(image):
    ori_image = image.copy()
    h, w = image.shape[:2]
    scale = 800 / min(h, w)
    if max(h, w) * scale > 1536:
        scale = 1536 / max(h, w)
    target_h = int(h * scale)
    target_w = int(w * scale)
    image = cv2.resize(image, (target_w, target_h))
    image = F.normalize(F.to_tensor(image), [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    return image, ori_image

## Tracking functions and classes

In [4]:
import torch
from MeMOTR.structures.track_instances import TrackInstances


def filter_by_score(tracks: TrackInstances, thresh: float = 0.7):
    keep = torch.max(tracks.scores, dim=-1).values > thresh
    return tracks[keep]

def filter_by_area(tracks: TrackInstances, thresh: int = 100):
    assert len(tracks.area) == len(tracks.ids), f"Tracks' 'area' should have the same dim with 'ids'"
    keep = tracks.area > thresh
    return tracks[keep]

## Plot functions

In [5]:
import numpy as np
import cv2


def get_color(idx):
    idx = idx * 3
    color = ((37 * idx) % 255, (17 * idx) % 255, (29 * idx) % 255)

    return color

def plot_tracking(image, tlwhs, obj_ids, scores=None, frame_id=0, fps=0., ids2=None,prompt=""):
    # Thanks to https://github.com/noahcao/OC_SORT
    im = np.ascontiguousarray(np.copy(image))
    im_h, im_w = im.shape[:2]

    top_view = np.zeros([im_w, im_w, 3], dtype=np.uint8) + 255
    text_scale = 2
    text_thickness = 2
    line_thickness = 3

    radius = max(5, int(im_w/140.))
    cv2.putText(im, 'frame: %d fps: %.2f num: %d' % (frame_id, fps, len(tlwhs)),
                (0, int(10 * text_scale)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), thickness=2)
    cv2.putText(im, "prompt: {}".format(prompt),  (0, int(20 * text_scale)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), thickness=2)

    for i, tlwh in enumerate(tlwhs):
        x1, y1, w, h = tlwh
        intbox = tuple(map(int, (x1, y1, x1 + w, y1 + h)))
        obj_id = int(obj_ids[i])
        id_text = '{}'.format(int(obj_id))
        if ids2 is not None:
            id_text = id_text + ', {}'.format(int(ids2[i]))
        color = get_color(abs(obj_id))
        cv2.rectangle(im, intbox[0:2], intbox[2:4], color=color, thickness=line_thickness)
        cv2.putText(im, id_text, (intbox[0], intbox[1]), cv2.FONT_HERSHEY_PLAIN, text_scale, (0, 0, 255),
                    thickness=text_thickness)
        
    return im

## Demo (inference) functions

In [6]:
import cv2
import time
import os.path
import sys
import numpy as np
import torch
from typing import List
from MeMOTR.models import build_model
from MeMOTR.models.utils import load_checkpoint, get_model,cut_image_from_bbox2
from MeMOTR.models.runtime_tracker import RuntimeTracker
from MeMOTR.utils.utils import yaml_to_dict, is_distributed, distributed_world_size, distributed_rank, inverse_sigmoid
from MeMOTR.utils.nested_tensor import tensor_list_to_nested_tensor
from MeMOTR.utils.box_ops import box_cxcywh_to_xyxy
from MeMOTR.structures.track_instances import TrackInstances
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
from MeMOTR.models.utils import checkBoxPrompt
CLIPModelQuery=CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
CLIPProcessorQuery = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def demo_processing(
        model_path: str,
        config_path: str,
        video_path: str,
):
    config = yaml_to_dict(config_path)
    model = build_model(config)
    load_checkpoint(
        model=model,
        path=model_path
    )
    model.eval()
    print("Model loaded.")
    current_time = time.localtime()
    cap = cv2.VideoCapture(video_path)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fps = cap.get(cv2.CAP_PROP_FPS)
    timestamp = time.strftime("%Y_%m_%d_%H_%M_%S", current_time)
    save_folder = os.path.join("C:\\Users\\phamp\\Desktop\MeMOTR-thesis\\MeMOTR\\output_", timestamp)
    save_path = os.path.join(save_folder, "output.mp4")
    os.makedirs(save_folder, exist_ok=True)
    vid_writer = cv2.VideoWriter(
        save_path, cv2.VideoWriter_fourcc(*'MP4V'), fps, (int(width), int(height))
    )
    print((int(width), int(height)))

    result_score_thresh = 0.5

    timer = Timer()
    frame_id = 0


    tracks = [TrackInstances(
        hidden_dim=model.hidden_dim,
        num_classes=model.num_classes,
        use_dab=config["USE_DAB"],
    ).to("cuda")]
    tracker = RuntimeTracker(
        det_score_thresh=0.5, 
        track_score_thresh=0.01,
        miss_tolerance=30,
        use_motion=False,
        motion_min_length=0, 
        motion_max_length=0,
        visualize=False, 
        use_dab=config["USE_DAB"],
    )

    with torch.no_grad():
        while True:
            prompt="black shirt"
            if  frame_id < 120 or frame_id > 240:
                prompt= "white shirt"
            
            if frame_id % 20 == 0:
                print('Processing frame {} ({:.2f} fps)'.format(frame_id, 1. / max(1e-5, timer.average_time)))
            ret_val, ret_frame = cap.read()
            # online_im = ret_frame
            if ret_val:
                image = process_image(ret_frame)
                frame = tensor_list_to_nested_tensor([image[0]]).to("cuda")
                timer.tic()
                res = model(frame=frame, tracks=tracks)
                previous_tracks, new_tracks = tracker.update(
                    model_outputs=res,
                    tracks=tracks
               
                )
                # new_tracks=process_track(new_tracks, ret_frame, prompt, width, height)
                # if(frame_id==0):
                    # previous_tracks = process_track(previous_tracks, ret_frame, prompt, width, height)
                # print(previous_tracks[0])
                tracks: List[TrackInstances] = model.postprocess_single_frame(previous_tracks, new_tracks, 
                                                                              None)
                
                tracks_result = tracks[0].to(torch.device("cpu"))
                # ori_h, ori_w = ori_image.shape[1], ori_image.shape[2]
                ori_h, ori_w = height, width
                # box = [x, y, w, h]
                tracks_result.area = tracks_result.boxes[:, 2] * ori_w * \
                                     tracks_result.boxes[:, 3] * ori_h
                tracks_result = filter_by_score(tracks_result, thresh=result_score_thresh)
                tracks_result = filter_by_area(tracks_result)
                # to xyxy:
                tracks_result.boxes = box_cxcywh_to_xyxy(tracks_result.boxes)
                tracks_result.boxes = (tracks_result.boxes * torch.as_tensor([ori_w, ori_h, ori_w, ori_h], dtype=torch.float))
                # allImages = cut_image_from_bbox2(ret_frame, tracks_result.boxes)
                
                # if len(allImages)!=0:
                #     with torch.no_grad():
                #         inputs = CLIPProcessorQuery(text=[prompt], 
                #                         images=allImages, return_tensors="pt", padding=True)
                #         outputs = CLIPModelQuery(**inputs)
                #         logits_per_image = outputs.logits_per_image # this is the image-text similarity score
                #         stack_pos=0
                #         print(logits_per_image)
                #         for i,pro in enumerate(logits_per_image):
                #             if pro.item()<23:
                #                 tracks_result.removePosition(i-stack_pos)
                #                 stack_pos+=1
                mask_check_box=checkBoxPrompt(ret_frame,tracks_result.boxes,ret_frame.shape[1],ret_frame.shape[0],prompt,23.5).flatten().tolist()
                online_tlwhs, online_ids = [], []
                tracks_result = tracks_result[mask_check_box]
                print("Number of tracks: ", len(tracks_result))
                for i in range(len(tracks_result)):
                    x1, y1, x2, y2 = tracks_result.boxes[i].tolist()
                    w, h = x2 - x1, y2 - y1
                    online_tlwhs.append([x1, y1, w, h])
                    online_ids.append(tracks_result.ids[i].item())
                timer.toc()
                if len(online_tlwhs) > 0:
                    online_im = plot_tracking(
                        ret_frame, online_tlwhs, online_ids, frame_id=frame_id + 1, fps=1. / timer.average_time,prompt=prompt
                    )
                else:
                    online_im = ret_frame
                vid_writer.write(online_im)
                ch = cv2.waitKey(1)
                if ch == 27 or ch == ord("q") or ch == ord("Q"):
                    break
            else:
                break
            frame_id += 1
    return os.path.join(save_folder, "output.mp4")

c:\Users\phamp\miniforge3\envs\PY10CU118\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\phamp\miniforge3\envs\PY10CU118\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Other functions

In [7]:
class Timer(object):
    """A simple timer."""
    def __init__(self):
        self.total_time = 0.
        self.calls = 0
        self.start_time = 0.
        self.diff = 0.
        self.average_time = 0.

        self.duration = 0.

    def tic(self):
        # using time.time instead of time.clock because time time.clock
        # does not normalize for multithreading
        self.start_time = time.time()

    def toc(self, average=True):
        self.diff = time.time() - self.start_time
        self.total_time += self.diff
        self.calls += 1
        self.average_time = self.total_time / self.calls
        if average:
            self.duration = self.average_time
        else:
            self.duration = self.diff
        return self.duration

# DEMO

## Select demo mp4

In [8]:
video_path = ".\\test_occlusion2_test.mp4"                                  # you should give me a mp4 file path
# config_path = ".\\MeMOTR\\configs\\train_mot17.yaml"          # the config path of model
# model_path = ".\\MeMOTR\\outputs\\memotr_mot17\\memotr_mot17.pth"       # the model checkpoint path
                        # play the video
config_path = ".\\MeMOTR\\configs\\train_bdd100k.yaml"          # the config path of model
model_path = ".\\MeMOTR\\outputs\\memotr_bdd100k\\memotr_bdd100k.pth"       # the model checkpoint path

## Run Demo

In [9]:
output_path = demo_processing(
    model_path=model_path,
    config_path=config_path,
    video_path=video_path,
)

Model loaded.
(1280, 720)
Processing frame 0 (100000.00 fps)


c:\Users\phamp\miniforge3\envs\PY10CU118\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tensor([[24.9631, 22.0777, 20.6321, 24.3289, 20.7526, 20.7043, 21.0024]])
Number of tracks:  2
tensor([[24.7706, 22.3189, 20.6947, 25.5627, 19.9788, 21.0684, 20.6701]])
Number of tracks:  2
tensor([[24.9327, 22.3621, 20.6177, 25.4875, 19.8099, 21.2672, 22.8406]])
Number of tracks:  2
tensor([[23.9451, 21.2146, 20.7087, 25.5335, 19.1715, 21.6359, 22.5196]])
Number of tracks:  2
tensor([[25.2650, 21.4754, 19.6212, 25.1367, 20.3086, 20.6951, 22.7366, 19.1529]])
Number of tracks:  2
tensor([[23.5067, 20.8872, 19.9329, 25.0364, 20.9375, 20.7605, 21.1679, 20.0054]])
Number of tracks:  2
tensor([[25.3341, 21.7271, 19.3693, 23.1180, 20.9269, 21.3180, 21.8905, 21.2199]])
Number of tracks:  1
tensor([[24.0233, 21.0631, 21.2861, 23.9523, 20.0367, 22.4702, 21.3904, 19.9859]])
Number of tracks:  2
tensor([[24.2762, 21.4054, 23.5703, 25.3321, 21.2024, 22.2120, 21.1038, 20.5047]])
Number of tracks:  3
tensor([[24.8323, 22.1116, 21.0526, 25.0524, 21.3597, 23.1499, 20.5978, 20.1012]])
Number of tracks: